In [1]:
import pandas as pd

### Read in price dataset and ready it for merge

In [2]:
df_prices = pd.read_csv("price.csv")

In [3]:
df_prices.columns=["City","Date","Product","CRP"]
df_prices.reset_index(inplace=True)
df_prices.drop(labels="index",axis=1,inplace=True)

## Ready Price Dataset for merge

In [4]:
# Convert string dates to datetime objects
df_prices['Date']= pd.to_datetime(df_prices['Date'])
df_prices.sort_values(by="Date", inplace=True)

df_prices.columns=["City","Date","Product","CRP"]
df_prices.reset_index(inplace=True)
df_prices.drop(labels="index",axis=1,inplace=True)


In [5]:
## Add one day so we can merge 
df_prices["Date"] = df_prices["Date"] + pd.to_timedelta(1, unit='d')

## Train dataset

In [6]:
df_train = pd.read_csv("train.csv")

In [7]:
df_train.head()

,Customer,Area,City Code,Billing Date,Product,Plant,Litres
0,10048261,Karadeniz Bölgesi,883,2014-01-01,Benzin,T102,0.0
1,10048261,Karadeniz Bölgesi,883,2014-01-01,Benzin,T155,0.0
2,10048261,Karadeniz Bölgesi,883,2014-01-01,Motorin,T102,0.0
3,10048261,Karadeniz Bölgesi,883,2014-01-01,Motorin,T155,0.0
4,10048261,Karadeniz Bölgesi,883,2014-01-02,Motorin,T155,16685.0


- Sum up all orders for a week
    I want to have a single entry summing up the orders placed for each week
    Will sum up orders for the same fuel across different plants

In [8]:
# rename columns for ease
df_train.columns=["Customer", "Area", "City", "Date", "Product", "Plant", "Litres"]

# convert to datetime, subtract 7 days so we can get the total for the following week
df_train['Date'] = pd.to_datetime(df_train['Date']) - pd.to_timedelta(7, unit='d')
# Sort
df_train.sort_values(by=["Date", "Customer", "Product"], inplace=True)

In [9]:
df_train.head()

,Customer,Area,City,Date,Product,Plant,Litres
0,10048261,Karadeniz Bölgesi,883,2013-12-25,Benzin,T102,0.0
1,10048261,Karadeniz Bölgesi,883,2013-12-25,Benzin,T155,0.0
2,10048261,Karadeniz Bölgesi,883,2013-12-25,Motorin,T102,0.0
3,10048261,Karadeniz Bölgesi,883,2013-12-25,Motorin,T155,0.0
2283,10048264,Karadeniz Bölgesi,883,2013-12-25,Motorin,T105,0.0


- Below I sum up orders across week, and drop the plant code

In [10]:
df_t = df_train.groupby(['Customer', "Area","City","Product", pd.Grouper(key='Date', freq='W-MON')])['Litres'].sum().reset_index().sort_values('Date')

In [11]:
#sort values to inspect
df_t.sort_values(by=["Date","Customer", "Product"], inplace=True)

In [12]:
df_t.head(5)

,Customer,Area,City,Product,Date,Litres
0,10048261,Karadeniz Bölgesi,883,Benzin,2013-12-30,5040.0
256,10048261,Karadeniz Bölgesi,883,Motorin,2013-12-30,28712.0
685,10048263,Karadeniz Bölgesi,966,Motorin,2013-12-30,34082.0
899,10048264,Karadeniz Bölgesi,883,Benzin,2013-12-30,3608.0
1069,10048264,Karadeniz Bölgesi,883,Motorin,2013-12-30,21325.0


In [13]:
# Remove first week because the data for it is incomplete
df_t = df_t[df_t["Date"] != "2013-12-30" ]

In [14]:
# reassign to original df
df_train = df_t.copy()

### Get Test Dataset

In [36]:
df_test = pd.read_csv("test.csv")
df_test["Date"] = pd.to_datetime(df_test['Week Start Date']) 
df_test.drop(labels=["index","Date"],axis=1,inplace=True)

In [38]:
df_test.columns = ["Customer", "Area", "City","Product","Date"]

In [40]:
df_test.head()

,Customer,Area,City,Product,Date
0,10048261,Karadeniz Bölgesi,883,Motorin,2019-01-07
1,10048261,Karadeniz Bölgesi,883,Motorin,2019-01-14
2,10048261,Karadeniz Bölgesi,883,Motorin,2019-01-21
3,10048261,Karadeniz Bölgesi,883,Motorin,2019-01-28
4,10048261,Karadeniz Bölgesi,883,Motorin,2019-02-04


In [45]:
df = pd.concat([df_train,df_test])
df["Date"] = pd.to_datetime(df['Date']) 

## Combine price and training datasets
 - For the week starting on 2014-01-06(Monday), the price data will be the price on 2014-01-05(Sunday)

In [47]:
# Merge datasets
df_merged = df.merge(df_prices,how="left")
# Rename column names for clarity
df_merged.rename(columns={"CRP":"Price(t-1)"}, inplace=True)


In [49]:
df_merged

,Customer,Area,City,Product,Date,Litres,Price(t-1)
0,10048261,Karadeniz Bölgesi,883,Benzin,2014-01-06,4242.0,5.07
1,10048261,Karadeniz Bölgesi,883,Motorin,2014-01-06,29017.0,4.40
2,10048263,Karadeniz Bölgesi,966,Benzin,2014-01-06,4572.0,5.04
3,10048263,Karadeniz Bölgesi,966,Motorin,2014-01-06,30681.0,4.34
4,10048264,Karadeniz Bölgesi,883,Motorin,2014-01-06,25009.0,4.40
...,...,...,...,...,...,...,...
587421,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-02,NaN,7.14
587422,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-09,NaN,7.14
587423,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-16,NaN,7.14
587424,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-23,NaN,7.14


In [50]:
# Check for nan values
df_merged["Price(t-1)"].isna().any()

True

In [51]:
df_merged["Price(t-1)"].isna().sum()

8721

#### Fill in the Nan values in Price t-1

In [52]:
df_merged.head()

,Customer,Area,City,Product,Date,Litres,Price(t-1)
0,10048261,Karadeniz Bölgesi,883,Benzin,2014-01-06,4242.0,5.07
1,10048261,Karadeniz Bölgesi,883,Motorin,2014-01-06,29017.0,4.40
2,10048263,Karadeniz Bölgesi,966,Benzin,2014-01-06,4572.0,5.04
3,10048263,Karadeniz Bölgesi,966,Motorin,2014-01-06,30681.0,4.34
4,10048264,Karadeniz Bölgesi,883,Motorin,2014-01-06,25009.0,4.40


In [53]:
temp = df_merged.groupby(['Date', 'Product'])[['Price(t-1)']].mean().reset_index()

In [54]:
temp.rename(columns={'Price(t-1)': 'Price(t-1)_mean'}, inplace=True)

In [55]:
df_temp = pd.merge(df_merged, temp, how='left', on=['Date', 'Product'])

In [56]:
df_temp.loc[df_temp['Price(t-1)'].isna(),'Price(t-1)'] = df_temp['Price(t-1)_mean']

In [57]:
df_temp

,Customer,Area,City,Product,Date,Litres,Price(t-1),Price(t-1)_mean
0,10048261,Karadeniz Bölgesi,883,Benzin,2014-01-06,4242.0,5.07,5.039854
1,10048261,Karadeniz Bölgesi,883,Motorin,2014-01-06,29017.0,4.40,4.346342
2,10048263,Karadeniz Bölgesi,966,Benzin,2014-01-06,4572.0,5.04,5.039854
3,10048263,Karadeniz Bölgesi,966,Motorin,2014-01-06,30681.0,4.34,4.346342
4,10048264,Karadeniz Bölgesi,883,Motorin,2014-01-06,25009.0,4.40,4.346342
...,...,...,...,...,...,...,...,...
587421,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-02,NaN,7.14,6.963877
587422,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-09,NaN,7.14,6.963877
587423,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-16,NaN,7.14,6.963877
587424,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-23,NaN,7.14,6.965134


In [58]:
# show none values
df_temp[df_temp.isna().any(axis=1)]

,Customer,Area,City,Product,Date,Litres,Price(t-1),Price(t-1)_mean
94677,10048261,Karadeniz Bölgesi,883,Benzin,2015-01-05,4215.0,NaN,NaN
94678,10048261,Karadeniz Bölgesi,883,Motorin,2015-01-05,18791.0,NaN,NaN
94679,10048263,Karadeniz Bölgesi,966,Motorin,2015-01-05,23570.0,NaN,NaN
94680,10048264,Karadeniz Bölgesi,883,Benzin,2015-01-05,3609.0,NaN,NaN
94681,10048264,Karadeniz Bölgesi,883,Motorin,2015-01-05,21576.0,NaN,NaN
...,...,...,...,...,...,...,...,...
587421,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-02,NaN,7.14,6.963877
587422,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-09,NaN,7.14,6.963877
587423,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-16,NaN,7.14,6.963877
587424,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-23,NaN,7.14,6.965134


In [64]:
df_temp[df_temp['Price(t-1)'].isna()].Date.unique()

array(['2015-01-05T00:00:00.000000000', '2015-01-12T00:00:00.000000000',
       '2015-02-09T00:00:00.000000000', '2015-03-09T00:00:00.000000000'],
      dtype='datetime64[ns]')

Manually impute prices from the preceeding and next week

In [65]:
b1 = df_temp[(df_temp["Date"] == "2014-12-29") & (df_temp["Product"] == "Benzin")]["Price(t-1)_mean"].mean()
b2= df_temp[(df_temp["Date"] == "2015-01-19") & (df_temp["Product"] == "Benzin")]["Price(t-1)_mean"].mean()

b3= df_temp[(df_temp["Date"] == "2015-02-02") & (df_temp["Product"] == "Benzin")]["Price(t-1)_mean"].mean()
b4= df_temp[(df_temp["Date"] == "2015-02-16") & (df_temp["Product"] == "Benzin")]["Price(t-1)_mean"].mean()

b5= df_temp[(df_temp["Date"] == "2015-03-02") & (df_temp["Product"] == "Benzin")]["Price(t-1)_mean"].mean()
b6= df_temp[(df_temp["Date"] == "2015-03-16") & (df_temp["Product"] == "Benzin")]["Price(t-1)_mean"].mean()

print(b1,b2,b3,b4,b5,b6)

d1 = df_temp[(df_temp["Date"] == "2014-12-29") & (df_temp["Product"] == "Motorin")]["Price(t-1)_mean"].mean()
d2= df_temp[(df_temp["Date"] == "2015-01-19") & (df_temp["Product"] == "Motorin")]["Price(t-1)_mean"].mean()

d3 = df_temp[(df_temp["Date"] == "2015-02-02") & (df_temp["Product"] == "Motorin")]["Price(t-1)_mean"].mean()
d4 = df_temp[(df_temp["Date"] == "2015-02-16") & (df_temp["Product"] == "Motorin")]["Price(t-1)_mean"].mean()

d5= df_temp[(df_temp["Date"] == "2015-03-02") & (df_temp["Product"] == "Motorin")]["Price(t-1)_mean"].mean()
d6= df_temp[(df_temp["Date"] == "2015-03-16") & (df_temp["Product"] == "Motorin")]["Price(t-1)_mean"].mean()

print(d1,d2,d3,d4,d5,d6)

4.2814469453377315 4.0857515527950445 4.280414129110803 4.389144215530904 4.191827338129465 4.477158671586621
3.793650000000047 3.553205128205157 3.7930486486485675 3.9143798882681846 3.711831735889189 3.8571668415529596


In [66]:
df_temp.loc[(df_temp["Date"] == "2015-01-05") & (df_temp["Product"] == "Benzin"), "Price(t-1)"] = (b1 + b2) /2
df_temp.loc[(df_temp["Date"] == "2015-01-12") & (df_temp["Product"] == "Benzin"), "Price(t-1)"] = (b1 + b2) /2

df_temp.loc[(df_temp["Date"] == "2015-02-09") & (df_temp["Product"] == "Benzin"), "Price(t-1)"] = (b3 + b4) /2
df_temp.loc[(df_temp["Date"] == "2015-03-09") & (df_temp["Product"] == "Benzin"), "Price(t-1)"] = (b5 + b6) /2

#diesels
df_temp.loc[(df_temp["Date"] == "2015-01-05") & (df_temp["Product"] == "Motorin"), "Price(t-1)"] = (d1 + d2) /2
df_temp.loc[(df_temp["Date"] == "2015-01-12") & (df_temp["Product"] == "Motorin"), "Price(t-1)"] = (d1 + d2) /2

df_temp.loc[(df_temp["Date"] == "2015-02-09") & (df_temp["Product"] == "Motorin"), "Price(t-1)"] = (d3 + d4) /2
df_temp.loc[(df_temp["Date"] == "2015-03-09") & (df_temp["Product"] == "Motorin"), "Price(t-1)"] = (d5 + d6) /2


### Check how successfull the filling of data was

In [67]:
df_temp['Price(t-1)'].isna().any()

False

### Finish Merge

In [77]:
# Reassign to df_train
df = df_temp.copy()
df.drop(labels="Price(t-1)_mean",axis=1,inplace=True)
df.head()

,Customer,Area,City,Product,Date,Litres,Price(t-1)
0,10048261,Karadeniz Bölgesi,883,Benzin,2014-01-06,4242.0,5.07
1,10048261,Karadeniz Bölgesi,883,Motorin,2014-01-06,29017.0,4.40
2,10048263,Karadeniz Bölgesi,966,Benzin,2014-01-06,4572.0,5.04
3,10048263,Karadeniz Bölgesi,966,Motorin,2014-01-06,30681.0,4.34
4,10048264,Karadeniz Bölgesi,883,Motorin,2014-01-06,25009.0,4.40


Drop city

In [78]:
df.drop(labels=["City"],axis=1,inplace=True)

In [79]:
df.head()

,Customer,Area,Product,Date,Litres,Price(t-1)
0,10048261,Karadeniz Bölgesi,Benzin,2014-01-06,4242.0,5.07
1,10048261,Karadeniz Bölgesi,Motorin,2014-01-06,29017.0,4.40
2,10048263,Karadeniz Bölgesi,Benzin,2014-01-06,4572.0,5.04
3,10048263,Karadeniz Bölgesi,Motorin,2014-01-06,30681.0,4.34
4,10048264,Karadeniz Bölgesi,Motorin,2014-01-06,25009.0,4.40


In [80]:
df.isna().any()
# we expect litres to be nan for 2019 data

Customer      False
Area          False
Product       False
Date          False
Litres         True
Price(t-1)    False
dtype: bool

In [81]:
## Rename Column for clarity
df.rename(columns={"Date":"WeekStarting"},inplace=True)

# Add in external data

### Car data

In [82]:
df_cars = pd.read_csv("CarRegistrationDataReady.csv")

In [83]:
df_cars.columns = ["WeekStarting", "Area","NewCars"]
df_cars["WeekStarting"]  = pd.to_datetime(df_cars['WeekStarting'])
df_cars

,WeekStarting,Area,NewCars
0,2014-01-06,Akdeniz Bölgesi,12007
1,2014-01-13,Akdeniz Bölgesi,12007
2,2014-01-20,Akdeniz Bölgesi,12007
3,2014-01-27,Akdeniz Bölgesi,12007
4,2014-02-03,Akdeniz Bölgesi,9999
...,...,...,...
2186,2019-12-02,İç Anadolu Bölgesi,9058
2187,2019-12-09,İç Anadolu Bölgesi,9058
2188,2019-12-16,İç Anadolu Bölgesi,9058
2189,2019-12-23,İç Anadolu Bölgesi,9058


In [84]:
df_merged = df.merge(df_cars,how="left")

In [85]:
# Check one area
df_merged[df_merged["Area"]=="Karadeniz Bölgesi"]

,Customer,Area,Product,WeekStarting,Litres,Price(t-1),NewCars
0,10048261,Karadeniz Bölgesi,Benzin,2014-01-06,4242.0,5.07,5015
1,10048261,Karadeniz Bölgesi,Motorin,2014-01-06,29017.0,4.40,5015
2,10048263,Karadeniz Bölgesi,Benzin,2014-01-06,4572.0,5.04,5015
3,10048263,Karadeniz Bölgesi,Motorin,2014-01-06,30681.0,4.34,5015
4,10048264,Karadeniz Bölgesi,Motorin,2014-01-06,25009.0,4.40,5015
...,...,...,...,...,...,...,...
587109,12690825,Karadeniz Bölgesi,Benzin,2019-12-02,NaN,7.00,3598
587110,12690825,Karadeniz Bölgesi,Benzin,2019-12-09,NaN,7.00,3598
587111,12690825,Karadeniz Bölgesi,Benzin,2019-12-16,NaN,7.00,3598
587112,12690825,Karadeniz Bölgesi,Benzin,2019-12-23,NaN,7.00,3598


In [86]:
df = df_merged.copy()

### Holiday Data

In [87]:
df_holidays = pd.read_csv("HolidayDataReady.csv")

In [88]:
df_holidays.columns = ["WeekStarting", "WeekendHolidays","WeekdayHolidays"]
df_holidays["WeekStarting"]  = pd.to_datetime(df_holidays['WeekStarting'])
df_holidays.head()

,WeekStarting,WeekendHolidays,WeekdayHolidays
0,2013-12-30,0,1
1,2014-04-21,0,1
2,2014-04-28,0,1
3,2014-05-19,0,1
4,2014-07-21,1,0


In [89]:
df_merged = df.merge(df_holidays,how="left")

In [90]:
df = df_merged.copy()

### GDP Growth

In [91]:
df_gdp = pd.read_csv("GDPDataReady.csv")

In [92]:
df_gdp.columns = ["WeekStarting", "GDPGrowth%"]
df_gdp["WeekStarting"]  = pd.to_datetime(df_gdp['WeekStarting'])
df_gdp.head()

,WeekStarting,GDPGrowth%
0,2014-01-06,0.044
1,2014-01-13,0.044
2,2014-01-20,0.044
3,2014-01-27,0.044
4,2014-02-03,0.044


In [93]:
df_merged = df.merge(df_gdp,how="left")

In [94]:
df = df_merged.copy()

## InflationData

In [95]:
df_inflation = pd.read_csv("InflationDataReady.csv")

In [96]:
df_inflation.columns = ["WeekStarting", "MoMInflation%"]
df_inflation["WeekStarting"]  = pd.to_datetime(df_inflation['WeekStarting'])
df_inflation.head()

,WeekStarting,MoMInflation%
0,2014-01-06,0.01
1,2014-01-13,0.46
2,2014-01-20,0.46
3,2014-01-27,0.46
4,2014-02-03,0.46


In [97]:
df_merged = df.merge(df_inflation,how="left")

In [98]:
df = df_merged.copy()

### USD TRY

In [99]:
df_usd = pd.read_csv("USDDataReady.csv")

In [100]:
df_usd.columns = ["WeekStarting", "WoWUSDChange%"]
df_usd["WeekStarting"]  = pd.to_datetime(df_usd['WeekStarting'])
df_usd.head()

,WeekStarting,WoWUSDChange%
0,2014-01-06,-0.62%
1,2014-01-13,3.08%
2,2014-01-20,4.47%
3,2014-01-27,"-3,04%"
4,2014-02-03,"-1,80%"


In [102]:
df_merged = df.merge(df_usd,how="left")

In [103]:
df = df_merged.copy()

In [104]:
df

,Customer,Area,Product,WeekStarting,Litres,Price(t-1),NewCars,WeekendHolidays,WeekdayHolidays,GDPGrowth%,MoMInflation%,WoWUSDChange%
0,10048261,Karadeniz Bölgesi,Benzin,2014-01-06,4242.0,5.07,5015,NaN,NaN,0.044,0.01,-0.62%
1,10048261,Karadeniz Bölgesi,Motorin,2014-01-06,29017.0,4.40,5015,NaN,NaN,0.044,0.01,-0.62%
2,10048263,Karadeniz Bölgesi,Benzin,2014-01-06,4572.0,5.04,5015,NaN,NaN,0.044,0.01,-0.62%
3,10048263,Karadeniz Bölgesi,Motorin,2014-01-06,30681.0,4.34,5015,NaN,NaN,0.044,0.01,-0.62%
4,10048264,Karadeniz Bölgesi,Motorin,2014-01-06,25009.0,4.40,5015,NaN,NaN,0.044,0.01,-0.62%
...,...,...,...,...,...,...,...,...,...,...,...,...
587421,12739737,Doğu Anadolu Bölgesi,Benzin,2019-12-02,NaN,7.14,2004,NaN,NaN,-0.015,2.00,"0,52%"
587422,12739737,Doğu Anadolu Bölgesi,Benzin,2019-12-09,NaN,7.14,2004,NaN,NaN,0.009,2.00,"0,55%"
587423,12739737,Doğu Anadolu Bölgesi,Benzin,2019-12-16,NaN,7.14,2004,NaN,NaN,0.009,0.38,"2,21%"
587424,12739737,Doğu Anadolu Bölgesi,Benzin,2019-12-23,NaN,7.14,2004,NaN,NaN,0.009,0.38,"0,23%"


## Fill Nan Values

In [105]:
df.isna().any()

Customer           False
Area               False
Product            False
WeekStarting       False
Litres              True
Price(t-1)         False
NewCars            False
WeekendHolidays     True
WeekdayHolidays     True
GDPGrowth%         False
MoMInflation%      False
WoWUSDChange%      False
dtype: bool

In [110]:
df.WeekendHolidays = df.WeekendHolidays.fillna(int(0))
df.WeekdayHolidays = df.WeekdayHolidays.fillna(int(0))

In [116]:
# Change usd% to float by replacing , with . then removing % then casting into float
df["WoWUSDChange%"] = df["WoWUSDChange%"].str.replace(",", ".")
df["WoWUSDChange%"] = df["WoWUSDChange%"].str.rstrip('%').astype('float')
df["WoWUSDChange%"]

0        -0.62
1        -0.62
2        -0.62
3        -0.62
4        -0.62
          ... 
587421    0.52
587422    0.55
587423    2.21
587424    0.23
587425    0.37
Name: WoWUSDChange%, Length: 587426, dtype: float64

Add Season data

In [140]:
df = df.assign(Season=lambda x: x['WeekStarting'].apply(lambda y: 'Winter' if y.month in [12, 1, 2] else 'Summer' if y.month in [
               3, 4, 5] else 'Autumn' if y.month in [6, 7, 8] else 'Fall' if y.month in [9, 10, 11] else 'Unknown'))


In [119]:
# Categorize the columns (NOT one-hot encoding as pytorch will take care of this)
category_columns = ["Customer", "Area", "Product", "Season"]
df_train[category_columns] = df_train[category_columns].astype("category")
#df_train["Product"] = df_train["Product"].astype("category")

Check column types

In [123]:
# Check dtypes correctness here
df.dtypes

Customer                    int64
Area                       object
Product                    object
WeekStarting       datetime64[ns]
Litres                    float64
Price(t-1)                float64
NewCars                     int64
WeekendHolidays           float64
WeekdayHolidays           float64
GDPGrowth%                float64
MoMInflation%             float64
WoWUSDChange%             float64
dtype: object

Check NaN

In [125]:
df.isna().any()

Customer           False
Area               False
Product            False
WeekStarting       False
Litres              True
Price(t-1)         False
NewCars            False
WeekendHolidays    False
WeekdayHolidays    False
GDPGrowth%         False
MoMInflation%      False
WoWUSDChange%      False
dtype: bool

### Add time series data as new features
- Order of prious week (week t-1)
- Order of week t-2
- Order for the previous week last year ( week t-1, y-1)
- Order for the current week last year 

Other todo:
- Drop first year data before training
- Drop Customer	and Area (Emin olamadım, bunları da encode ettirmicez mi category olarak)
- Encode petrol/diesel, categorize ettim (Done)
- Add seasonality (Done)
- Change usd% to float (Done)



In [126]:
# Function to add week lag

def add_time_lag(week_lag,df):
    df_lag = df[["Customer","Area","Product","WeekStarting", "Litres"]].copy()
    df_lag["WeekStarting"] = df_lag["WeekStarting"] + pd.to_timedelta(week_lag,unit="w")
    df_lag.rename(columns={"Litres":f"Litres(w-{week_lag})"}, inplace=True)
    df = df_train.merge(df_lag,how="left")
    return df_train


Add 1 week time lag variable

In [127]:
df = add_time_lag(1,df)

Add 2 week time lag variable

In [128]:
df = add_time_lag(2,df)

Add 3 week time lag variable

In [129]:
df = add_time_lag(3,df)

Add 49 week time lag variable

In [130]:
df = add_time_lag(49,df)

Add 50 week time lag variable

In [131]:
df = add_time_lag(50,df)

Add 51 week time lag variable

In [132]:
df = add_time_lag(51,df)

Add 52 week time lag variable

In [133]:
df = add_time_lag(52,df)

In [134]:
df

,Customer,Area,Product,WeekStarting,Litres,Price(t-1),NewCars,WeekendHolidays,WeekdayHolidays,GDPGrowth%,MoMInflation%,WoWUSDChange%,Litres(w-1),Litres(w-2),Litres(w-3),Litres(w-49),Litres(w-50),Litres(w-51),Litres(w-52)
0,10048261,Karadeniz Bölgesi,Benzin,2014-01-06,4242.0,5.07,5015,0.0,0.0,0.044,0.01,-0.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10048261,Karadeniz Bölgesi,Motorin,2014-01-06,29017.0,4.40,5015,0.0,0.0,0.044,0.01,-0.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10048263,Karadeniz Bölgesi,Benzin,2014-01-06,4572.0,5.04,5015,0.0,0.0,0.044,0.01,-0.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10048263,Karadeniz Bölgesi,Motorin,2014-01-06,30681.0,4.34,5015,0.0,0.0,0.044,0.01,-0.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10048264,Karadeniz Bölgesi,Motorin,2014-01-06,25009.0,4.40,5015,0.0,0.0,0.044,0.01,-0.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587421,12739737,Doğu Anadolu Bölgesi,Benzin,2019-12-02,NaN,7.14,2004,0.0,0.0,-0.015,2.00,0.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
587422,12739737,Doğu Anadolu Bölgesi,Benzin,2019-12-09,NaN,7.14,2004,0.0,0.0,0.009,2.00,0.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN
587423,12739737,Doğu Anadolu Bölgesi,Benzin,2019-12-16,NaN,7.14,2004,0.0,0.0,0.009,0.38,2.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
587424,12739737,Doğu Anadolu Bölgesi,Benzin,2019-12-23,NaN,7.14,2004,0.0,0.0,0.009,0.38,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
df[["Litres(w-1)","Litres(w-2)","Litres(w-3)","Litres(w-49)","Litres(w-50)","Litres(w-51)","Litres(w-52)"] ] = df[["Litres(w-1)","Litres(w-2)","Litres(w-3)","Litres(w-49)","Litres(w-50)","Litres(w-51)","Litres(w-52)"] ].fillna(0)

Add week of year value

In [137]:
df["WeekNumber"] = df['WeekStarting'].dt.isocalendar().week.astype("category")

In [138]:
df.dtypes

Customer                    int64
Area                       object
Product                    object
WeekStarting       datetime64[ns]
Litres                    float64
Price(t-1)                float64
NewCars                     int64
WeekendHolidays           float64
WeekdayHolidays           float64
GDPGrowth%                float64
MoMInflation%             float64
WoWUSDChange%             float64
Litres(w-1)               float64
Litres(w-2)               float64
Litres(w-3)               float64
Litres(w-49)              float64
Litres(w-50)              float64
Litres(w-51)              float64
Litres(w-52)              float64
WeekNumber               category
dtype: object

In [142]:
#df_train.to_csv("TrainingDataWithExternalData.csv")
df.to_pickle("Training_TestData.pkl")